In [1]:
import requests
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime
import numpy as np
from scipy.optimize import curve_fit
import json

In [2]:
def fetch():
    df = pd.DataFrame(requests.get('https://covidtracking.com/api/states/daily').json())
#     df['dateChecked'] = pd.to_datetime(df['dateChecked'])
#     df['date'] = list(map(str, df['date']))
#     year, month, day = zip(*[list(map(int, (d[:4], d[4:6], d[6:]))) for d in df['date']])
#     df['date'] = [datetime(y, m, d) for y, m, d in zip(year, month, day)]
    totals = df.groupby('dateChecked').agg('sum')
    return totals

def regress(x_data, y_data):
    first_nonzero = 0
    while y_data[first_nonzero] == 0:
        first_nonzero += 1
    x_data = x_data[first_nonzero:]
    y_data = y_data[first_nonzero:]
    log_x_data = np.log(x_data)
    log_y_data = np.log(y_data)

    exp, coeff = np.polyfit(x_data, log_y_data, 1)

    y = np.exp(coeff) * np.exp(exp)**x_data
    print("Base -", np.exp(coeff), "r0 -", np.exp(exp))
    return np.exp(coeff), np.exp(exp)

def what_if(arr, day, amount_home):
    base, r0 = regress(np.arange(0, len(arr)), arr)
    vals = []
    total = base
    for i, val in enumerate(arr):
#         print(i, base * r0**i, total, base * r0**i - total)
        vals.append(total)
        if i == day:
            total *= r0
            total -= amount_home
        else:
            total *= r0
    return vals

In [3]:
def main():
    totals = fetch()

    base, r0 = regress(np.arange(0, len(totals)), totals['positive'])
    vals = what_if(totals['positive'], 5, 10)
    totals['expected_positive'] = [base * r0**i for i in range(len(totals))]
    totals['new_positive'] = vals
    mortality_rate = totals['death'][-1] / totals['positive'][-1]
    hosp_rate = totals['hospitalized'][-1] / totals['positive'][-1]
    totals['expected_death'] = mortality_rate * totals['expected_positive']
    totals['expected_hospitalized'] = hosp_rate * totals['expected_positive']
    totals['new_death'] = mortality_rate * totals['new_positive']
    totals['new_hospitalized'] = hosp_rate * totals['new_positive']
    totals['day'] = np.arange(0, len(totals))
    totals['positive_rate'] = totals['positive'] / (totals['positive'] + totals['negative'])

#     plt.plot(totals['day'], totals['expected'])
#     plt.plot(totals['day'], totals['new'])
    totals.to_dict('list')
    return totals

main()

Base - 122.79979394486433 r0 - 1.3545784853528842
Base - 122.79979394486433 r0 - 1.3545784853528842


/Users/anirudhkamath/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


,date,positive,negative,pending,hospitalized,death,total,expected_positive,new_positive,expected_death,new_death,day,positive_rate
dateChecked,,,,,,,,,,,,,
2020-03-04T21:00:00Z,282804256,118.0,748.0,103.0,0.0,0.0,969,122.799794,166.341959,1.594956,2.160493,0,0.136259
2020-03-05T21:00:00Z,484807320,176.0,953.0,197.0,0.0,0.0,1326,166.341959,225.323239,2.160493,2.926557,1,0.155890
2020-03-06T21:00:00Z,727211016,223.0,1571.0,458.0,0.0,0.0,2252,225.323239,305.218011,2.926557,3.964252,2,0.124303
2020-03-07T21:00:00Z,1030215657,341.0,1809.0,602.0,0.0,0.0,2752,305.218011,413.441752,3.964252,5.369890,3,0.158605
2020-03-08T20:00:00Z,1030215708,417.0,2335.0,347.0,0.0,0.0,3099,413.441752,560.039302,5.369890,7.273937,4,0.151526
2020-03-09T20:00:00Z,1030215759,584.0,3367.0,313.0,0.0,0.0,4264,560.039302,748.617189,7.273937,9.723237,5,0.147811
2020-03-10T20:00:00Z,1030215810,778.0,3807.0,469.0,0.0,0.0,5054,758.617189,1014.060738,9.853119,13.170887,6,0.169684
2020-03-11T20:00:00Z,1030215861,1053.0,5978.0,563.0,0.0,27.0,7617,1027.606523,1373.624858,13.346823,17.841000,7,0.149765
2020-03-12T20:00:00Z,1030215912,1315.0,7949.0,673.0,0.0,36.0,9966,1391.973687,1860.682680,18.079320,24.167035,8,0.141947


In [4]:
totals

NameError: name 'totals' is not defined